In [1]:
import os.path

from revscoring import Model


model_path = os.path.expanduser("~/editquality/models/enwiki.damaging.gradient_boosting.model")
sm = Model.load(open(model_path), error_on_env_check=False)

/home/awight/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Differences between the current environment and the environment in which the model was constructed environment were detected:
 - platform 'Linux-4.9.0-6-amd64-x86_64-with-debian-9.4' mismatch with original environment 'Linux-4.16.0-x86_64-with-debian-9.4'
 - revscoring_version '2.2.2' mismatch with original environment '2.2.5'
 - release '4.9.0-6-amd64' mismatch with original environment '4.16.0'
 - version '#1 SMP Debian 4.9.82-1+deb9u3 (2018-03-02)' mismatch with original environment '#1 SMP Wed Apr 18 14:02:11 PDT 2018'


In [2]:
import mwapi
from revscoring.extractors import api

extractor = api.Extractor(mwapi.Session("https://en.wikipedia.org", user_agent="ORES-LIME demo"))

In [37]:
import numpy as np

rev_to_score = 12345678
feature_values = np.array(list(extractor.extract(rev_to_score, sm.features)))
sm.score(feature_values)

{'prediction': False,
 'probability': {False: 0.9006672855880219, True: 0.09933271441197815}}

In [38]:
# FIXME: We would actually want to change the diff, though?
feature_values = np.array(list(extractor.extract(rev_to_score, sm.features, cache={"datasource.revision.text": "you"})))
sm.score(feature_values)

{'prediction': True,
 'probability': {False: 0.47502357204620316, True: 0.5249764279537968}}

In [21]:
# Load training set
from revscoring.utilities.util import read_observations

observations = list(read_observations(open(os.path.expanduser("~/editquality/datasets/enwiki.labeled_revisions.w_cache.20k_2015.json"))))

In [23]:
observations[0]

{'auto_labeled': False,
 'autolabel': {},
 'cache': {'feature.english.badwords.revision.diff.match_delta_decrease': 0,
  'feature.english.badwords.revision.diff.match_delta_increase': 0,
  'feature.english.badwords.revision.diff.match_delta_sum': 0,
  'feature.english.badwords.revision.diff.match_prop_delta_decrease': 0.0,
  'feature.english.badwords.revision.diff.match_prop_delta_increase': 0.0,
  'feature.english.badwords.revision.diff.match_prop_delta_sum': 0.0,
  'feature.english.dictionary.revision.diff.dict_word_delta_decrease': -1,
  'feature.english.dictionary.revision.diff.dict_word_delta_increase': 2,
  'feature.english.dictionary.revision.diff.dict_word_delta_sum': 1,
  'feature.english.dictionary.revision.diff.dict_word_prop_delta_decrease': -0.16666666666666666,
  'feature.english.dictionary.revision.diff.dict_word_prop_delta_increase': 0.11764705882352941,
  'feature.english.dictionary.revision.diff.dict_word_prop_delta_sum': -0.04901960784313725,
  'feature.english.dicti

In [50]:
# Transform training data into a numpy matrix.
import numpy as np

# FIXME: Don't cast booleans to float.

features = sorted(str(f) for f in sm.features)
train = np.array([np.array([o["cache"][k] for k in features]) for o in observations])

In [53]:
features

['feature.english.badwords.revision.diff.match_delta_decrease',
 'feature.english.badwords.revision.diff.match_delta_increase',
 'feature.english.badwords.revision.diff.match_delta_sum',
 'feature.english.badwords.revision.diff.match_prop_delta_decrease',
 'feature.english.badwords.revision.diff.match_prop_delta_increase',
 'feature.english.badwords.revision.diff.match_prop_delta_sum',
 'feature.english.dictionary.revision.diff.dict_word_delta_decrease',
 'feature.english.dictionary.revision.diff.dict_word_delta_increase',
 'feature.english.dictionary.revision.diff.dict_word_delta_sum',
 'feature.english.dictionary.revision.diff.dict_word_prop_delta_decrease',
 'feature.english.dictionary.revision.diff.dict_word_prop_delta_increase',
 'feature.english.dictionary.revision.diff.dict_word_prop_delta_sum',
 'feature.english.dictionary.revision.diff.non_dict_word_delta_decrease',
 'feature.english.dictionary.revision.diff.non_dict_word_delta_increase',
 'feature.english.dictionary.revision.

In [55]:
# TODO: Also explain as text, using our own explainer to run variations.

from sklearn.linear_model import LinearRegression
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    train,
    mode="classification",
    feature_names=features,
    class_names=["damaging", "not damaging"],
    discretize_continuous=True
)
exp = explainer.explain_instance(
    np.array(feature_values),
    sm.score,
    num_features=3,
    top_labels=3,
    model_regressor=LinearRegression()
)

ValueError: operands could not be broadcast together with shapes (1,395000) (79,) (1,395000) 